# CPR Framework: Complete Tutorial

This notebook provides a comprehensive, hands-on tutorial for the **Constraint Pressure Ratio (CPR) Framework** - a mathematical framework for predicting exploration behavior in constrained dynamical systems.

## Table of Contents
1. [Introduction](#introduction)
2. [Setup and Installation](#setup)
3. [Core Concepts](#concepts)
4. [Two Universality Classes](#universality)
5. [Making Predictions](#predictions)
6. [Regime Analysis](#regimes)
7. [Architecture Comparison](#architectures)
8. [Visualizations](#visualizations)
9. [Advanced Topics](#advanced)
10. [Real-World Applications](#applications)

<a id='introduction'></a>
## 1. Introduction

### What is the CPR Framework?

The **CPR (Constraint Pressure Ratio) Framework** predicts how constraints affect a system's ability to explore its state space. It was developed through analysis of 312 experiments across 27 different architectural configurations.

### Key Discovery: Two Universality Classes

The framework's major contribution is discovering that constraints fall into **two fundamental universality classes**, each requiring a different mathematical model:

- **Class I (Density-Based)**: Sigmoid phase transition behavior
- **Class II (Structure-Based)**: Linear complexity scaling

### Performance

- **Coverage**: 100% (27/27 architectures)
- **Accuracy**: R² > 0.95 for all models
- **Error**: RMSE < 0.05

<a id='setup'></a>
## 2. Setup and Installation

In [ ]:
# Import the CPR Framework
from implementation_complete import (
    predict_exploration,
    select_model,
    predict_from_cpr,
    predict_from_complexity,
    ARCHITECTURE_ADJUSTMENTS
)

# Standard libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from typing import Dict, List, Tuple

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ CPR Framework loaded successfully!")
print(f"✓ Available architectures: {len(ARCHITECTURE_ADJUSTMENTS)}")

<a id='concepts'></a>
## 3. Core Concepts

### 3.1 The CPR Metric

The **Constraint Pressure Ratio** quantifies the pressure that constraints place on a system:

$$CPR = \frac{n}{b^n}$$

Where:
- $n$ = system size (number of components)
- $b$ = base (number of states per component)

### 3.2 Exploration Efficiency

The **exploration efficiency** $E$ measures how effectively a system explores its available state space:

$$E \in [0, 1]$$

- $E \approx 0$: System is highly constrained, minimal exploration
- $E \approx 1$: System explores freely, maximal exploration

### 3.3 System Components

A system configuration consists of:
1. **Constraint**: The type of constraint applied (pattern_prohibition, sum_modulation, local_entropy)
2. **Mixing Method**: How constraints are combined (additive, multiplicative, triple_sum)
3. **Governor**: The exploration strategy (uniform_distribution, entropy_maximization, novelty_seeking)

In [ ]:
# Example: Calculate CPR for different system sizes
def calculate_cpr(n: int, b: int) -> float:
    """Calculate base CPR."""
    return n / (b ** n)

# Demonstrate CPR scaling
print("CPR scaling with system size (b=3):\n")
print(f"{'n':>3} | {'State Space':>15} | {'CPR':>15}")
print("-" * 40)

for n in [4, 6, 8, 10, 12, 14, 16]:
    state_space = 3 ** n
    cpr = calculate_cpr(n, 3)
    print(f"{n:3d} | {state_space:15,d} | {cpr:15.2e}")

print("\nNotice: CPR decreases exponentially as system size grows!")
print("This represents the exponential explosion of the state space.")

<a id='universality'></a>
## 4. Two Universality Classes

The framework's key discovery: constraints exhibit two fundamentally different behaviors.

### 4.1 Class I: Density-Based Constraints

**Constraints**: `sum_modulation`, `local_entropy`

**Behavior**: Sigmoid phase transition in log(CPR) space

**Model**:
$$E = \frac{L}{1 + e^{-k(\log_{10}(CPR_{adj}) - x_0)}}$$

**Parameters**:
- $L = 0.8513$ (upper asymptote)
- $k = 46.7978$ (steepness)
- $x_0 = -8.2999$ (critical point)

**Performance**: R² > 0.95, RMSE < 0.05

In [ ]:
# Demonstrate Class I: sum_modulation
print("CLASS I DEMONSTRATION: sum_modulation\n")
print(f"{'n':>3} | {'CPR':>12} | {'Predicted E':>12} | {'Regime':>15}")
print("-" * 60)

for n in range(6, 17, 2):
    result = predict_exploration(
        n=n, b=3,
        constraint="sum_modulation",
        mixing="additive",
        governor="uniform_distribution"
    )
    
    cpr = result['adjusted_cpr']
    e = result['prediction']
    
    # Determine regime
    if cpr > 1e-7:
        regime = "Constrained"
    elif cpr > 1e-9:
        regime = "Critical"
    else:
        regime = "Emergent"
    
    print(f"{n:3d} | {cpr:12.2e} | {e:12.4f} | {regime:>15}")

print("\n→ Notice the sigmoid transition through the regimes!")

### 4.2 Class II: Structure-Based Constraints

**Constraint**: `pattern_prohibition`

**Behavior**: Linear relationship with complexity

**Model**:
$$E = \left(\frac{C}{C_{max}}\right)^\alpha \times 10^\beta$$

**Parameters**:
- $C_{max} = 2.4467$ (maximum complexity)
- $\alpha = 0.90$ (power exponent)
- $\beta = -0.015$ (scale factor)

**Performance**: R² > 0.99, RMSE < 0.02

In [ ]:
# Demonstrate Class II: pattern_prohibition
print("CLASS II DEMONSTRATION: pattern_prohibition\n")
print(f"{'Complexity':>12} | {'Norm. C':>10} | {'Predicted E':>12}")
print("-" * 40)

for c in np.linspace(0.5, 2.4, 10):
    result = predict_exploration(
        n=12, b=3,
        constraint="pattern_prohibition",
        mixing="additive",
        governor="uniform_distribution",
        complexity=float(c)
    )
    
    norm_c = result['normalized_complexity']
    e = result['prediction']
    
    print(f"{c:12.3f} | {norm_c:10.4f} | {e:12.4f}")

print("\n→ Notice the nearly linear scaling with complexity!")

### 4.3 Visual Comparison of Universality Classes

In [ ]:
# Create side-by-side comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Class I: Sigmoid behavior
n_values = np.arange(6, 17)
e_values_class1 = []
cpr_values = []

for n in n_values:
    result = predict_exploration(
        n=int(n), b=3,
        constraint="sum_modulation",
        mixing="additive",
        governor="uniform_distribution"
    )
    e_values_class1.append(result['prediction'])
    cpr_values.append(result['adjusted_cpr'])

ax1.plot(n_values, e_values_class1, 'o-', linewidth=2, markersize=8, 
         color='#2E86AB', label='sum_modulation')
ax1.set_xlabel('System Size (n)', fontsize=12)
ax1.set_ylabel('Exploration Efficiency', fontsize=12)
ax1.set_title('Class I: Density-Based\n(Sigmoid Phase Transition)', 
              fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.set_ylim(-0.05, 0.9)

# Class II: Linear complexity scaling
complexity_values = np.linspace(0.5, 2.4, 30)
e_values_class2 = []

for c in complexity_values:
    result = predict_exploration(
        n=12, b=3,
        constraint="pattern_prohibition",
        mixing="additive",
        governor="uniform_distribution",
        complexity=float(c)
    )
    e_values_class2.append(result['prediction'])

ax2.plot(complexity_values, e_values_class2, 'o-', linewidth=2, markersize=6,
         color='#F18F01', label='pattern_prohibition')
ax2.set_xlabel('Complexity (C)', fontsize=12)
ax2.set_ylabel('Exploration Efficiency', fontsize=12)
ax2.set_title('Class II: Structure-Based\n(Linear Complexity Scaling)', 
              fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_ylim(-0.05, 0.9)

plt.tight_layout()
plt.savefig('universality_classes_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("Key Insight: The two classes exhibit fundamentally different mathematical behaviors!")

<a id='predictions'></a>
## 5. Making Predictions

The framework provides a simple interface for making predictions.

### 5.1 Basic Prediction

In [ ]:
# Example 1: Predict for a density-based constraint
result = predict_exploration(
    n=12,
    b=3,
    constraint="sum_modulation",
    mixing="additive",
    governor="uniform_distribution"
)

print("PREDICTION RESULTS")
print("="*50)
print(f"Model Used: {result['model']}")
print(f"Predicted Exploration: {result['prediction']:.4f}")
print(f"Base CPR: {result['base_cpr']:.6e}")
print(f"Adjustment Factor: {result['adjustment_factor']:.4f}")
print(f"Adjusted CPR: {result['adjusted_cpr']:.6e}")

In [ ]:
# Example 2: Predict for a structure-based constraint
result = predict_exploration(
    n=12,
    b=3,
    constraint="pattern_prohibition",
    mixing="multiplicative",
    governor="entropy_maximization",
    complexity=1.85
)

print("\nPREDICTION RESULTS")
print("="*50)
print(f"Model Used: {result['model']}")
print(f"Predicted Exploration: {result['prediction']:.4f}")
print(f"Complexity: {result.get('complexity', 'N/A')}")
print(f"Normalized Complexity: {result.get('normalized_complexity', 'N/A'):.4f}")

### 5.2 Model Selection

The framework automatically selects the appropriate model based on the constraint type.

In [ ]:
# Demonstrate automatic model selection
test_cases = [
    ("sum_modulation", None),
    ("local_entropy", None),
    ("pattern_prohibition", 1.85),
    ("pattern_prohibition", None)  # Missing complexity - will fallback
]

print("AUTOMATIC MODEL SELECTION\n")
print(f"{'Constraint':^25} | {'Complexity':^12} | {'Selected Model':^20}")
print("="*65)

for constraint, complexity in test_cases:
    model_info = select_model(constraint, complexity)
    comp_str = f"{complexity:.2f}" if complexity else "None"
    print(f"{constraint:^25} | {comp_str:^12} | {model_info['model']:^20}")
    print(f"  → {model_info['reason']}")
    print()

### 5.3 Batch Predictions

In [ ]:
# Batch predictions for multiple configurations
configurations = [
    {"n": 8, "b": 3, "constraint": "sum_modulation", 
     "mixing": "additive", "governor": "uniform_distribution"},
    {"n": 10, "b": 3, "constraint": "local_entropy", 
     "mixing": "multiplicative", "governor": "entropy_maximization"},
    {"n": 12, "b": 3, "constraint": "pattern_prohibition", 
     "mixing": "triple_sum", "governor": "novelty_seeking", "complexity": 1.95},
    {"n": 14, "b": 3, "constraint": "sum_modulation", 
     "mixing": "multiplicative", "governor": "novelty_seeking"},
]

print("BATCH PREDICTIONS\n")
for i, config in enumerate(configurations, 1):
    result = predict_exploration(**config)
    print(f"{i}. {config['constraint']} (n={config['n']})")
    print(f"   E = {result['prediction']:.4f} [{result['model']}]")
    print()

<a id='regimes'></a>
## 6. Regime Analysis

CPR systems exhibit three distinct regimes based on the CPR value.

### 6.1 The Three Regimes

1. **Constrained Regime** (High CPR > ~10⁻⁷)
   - Constraints dominate the system
   - Low exploration efficiency (E ≈ 0)
   - Small state space relative to constraints

2. **Critical Regime** (CPR ≈ 10⁻⁸ to 10⁻⁹)
   - Phase transition region
   - Rapid increase in exploration
   - Sensitive to system parameters

3. **Emergent Regime** (Low CPR < ~10⁻¹⁰)
   - System explores freely
   - High exploration efficiency (E → 0.85)
   - Large state space relative to constraints

In [ ]:
# Analyze regime transitions
print("REGIME TRANSITION ANALYSIS\n")
print(f"{'n':>3} | {'CPR':>15} | {'E':>8} | {'dE/dn':>10} | {'Regime':>15}")
print("="*70)

prev_e = None
for n in range(4, 18):
    result = predict_exploration(
        n=n, b=3,
        constraint="sum_modulation",
        mixing="additive",
        governor="uniform_distribution"
    )
    
    cpr = result['adjusted_cpr']
    e = result['prediction']
    
    # Calculate derivative (rate of change)
    if prev_e is not None:
        de_dn = e - prev_e
        de_str = f"{de_dn:+.4f}"
    else:
        de_str = "N/A"
    
    # Determine regime
    if cpr > 1e-7:
        regime = "CONSTRAINED"
    elif cpr > 1e-9:
        regime = "CRITICAL ⚠"
    else:
        regime = "EMERGENT"
    
    print(f"{n:3d} | {cpr:15.2e} | {e:8.4f} | {de_str:>10} | {regime:>15}")
    prev_e = e

print("\nNotice: Largest dE/dn occurs in the CRITICAL regime!")

### 6.2 Visualizing Regime Transitions

In [ ]:
# Create comprehensive regime visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))

# Collect data
n_values = np.arange(4, 18)
e_values = []
cpr_values = []

for n in n_values:
    result = predict_exploration(
        n=int(n), b=3,
        constraint="sum_modulation",
        mixing="additive",
        governor="uniform_distribution"
    )
    e_values.append(result['prediction'])
    cpr_values.append(result['adjusted_cpr'])

# Plot 1: E vs n with regime shading
ax1.plot(n_values, e_values, 'o-', linewidth=2, markersize=8, color='#2E86AB')
ax1.set_xlabel('System Size (n)', fontsize=11)
ax1.set_ylabel('Exploration Efficiency', fontsize=11)
ax1.set_title('Exploration vs System Size', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Add regime annotations
ax1.axhline(y=0.1, color='red', linestyle='--', alpha=0.3)
ax1.axhline(y=0.7, color='green', linestyle='--', alpha=0.3)
ax1.text(5, 0.05, 'Constrained', fontsize=9, color='red')
ax1.text(10, 0.4, 'Critical', fontsize=9, color='orange')
ax1.text(16, 0.75, 'Emergent', fontsize=9, color='green')

# Plot 2: E vs CPR (log scale)
ax2.semilogx(cpr_values, e_values, 'o-', linewidth=2, markersize=8, color='#A23B72')
ax2.set_xlabel('Adjusted CPR (log scale)', fontsize=11)
ax2.set_ylabel('Exploration Efficiency', fontsize=11)
ax2.set_title('Sigmoid Phase Transition', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, which='both')
ax2.invert_xaxis()

# Shade regimes
ax2.axvspan(1e-6, max(cpr_values), alpha=0.2, color='red', label='Constrained')
ax2.axvspan(1e-9, 1e-7, alpha=0.2, color='yellow', label='Critical')
ax2.axvspan(min(cpr_values), 1e-9, alpha=0.2, color='green', label='Emergent')
ax2.legend(loc='lower left')

# Plot 3: Rate of change
de_dn = np.diff(e_values)
ax3.plot(n_values[1:], de_dn, 'o-', linewidth=2, markersize=8, color='#F18F01')
ax3.set_xlabel('System Size (n)', fontsize=11)
ax3.set_ylabel('dE/dn (Rate of Change)', fontsize=11)
ax3.set_title('Exploration Growth Rate', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# Highlight maximum
max_idx = np.argmax(de_dn)
ax3.plot(n_values[1:][max_idx], de_dn[max_idx], 'r*', markersize=15, 
         label=f'Max at n={n_values[1:][max_idx]}')
ax3.legend()

# Plot 4: Log-log CPR scaling
ax4.loglog(n_values, cpr_values, 'o-', linewidth=2, markersize=8, color='#6A0572')
ax4.set_xlabel('System Size (n)', fontsize=11)
ax4.set_ylabel('Adjusted CPR', fontsize=11)
ax4.set_title('CPR Exponential Decay', fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig('regime_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

<a id='architectures'></a>
## 7. Architecture Comparison

The framework handles 27 different architectures (3 constraints × 3 mixing methods × 3 governors).

### 7.1 Architecture Components

**Constraints** (3):
- `pattern_prohibition`: Structural constraint on sequential patterns
- `sum_modulation`: Density constraint on state sums
- `local_entropy`: Density constraint on local randomness

**Mixing Methods** (3):
- `additive`: Simple addition of constraint effects
- `multiplicative`: Multiplicative combination
- `triple_sum`: Three-way summation

**Governors** (3):
- `uniform_distribution`: Uniform exploration strategy
- `entropy_maximization`: Maximize information
- `novelty_seeking`: Prefer novel states

In [ ]:
# Analyze all architectures
print(f"Total Architectures: {len(ARCHITECTURE_ADJUSTMENTS)}\n")

# Group by constraint
by_constraint = {}
for arch, factor in ARCHITECTURE_ADJUSTMENTS.items():
    parts = arch.split('_')
    if len(parts) >= 2:
        constraint = f"{parts[0]}_{parts[1]}"
        if constraint not in by_constraint:
            by_constraint[constraint] = []
        by_constraint[constraint].append((arch, factor))

print("Architectures by Constraint:\n")
for constraint, archs in sorted(by_constraint.items()):
    print(f"{constraint}:")
    print(f"  Count: {len(archs)}")
    factors = [f for _, f in archs]
    print(f"  Adjustment factors: {min(factors):.4f} to {max(factors):.4f}")
    print(f"  Mean: {np.mean(factors):.4f}, Std: {np.std(factors):.4f}")
    print()

### 7.2 Impact of Mixing Methods

In [ ]:
# Compare mixing methods for the same constraint and governor
print("MIXING METHOD COMPARISON\n")
print("Fixed: sum_modulation + uniform_distribution, n=12, b=3\n")
print(f"{'Mixing':^20} | {'Adj. Factor':^12} | {'Predicted E':^12}")
print("="*50)

for mixing in ["additive", "multiplicative", "triple_sum"]:
    result = predict_exploration(
        n=12, b=3,
        constraint="sum_modulation",
        mixing=mixing,
        governor="uniform_distribution"
    )
    print(f"{mixing:^20} | {result['adjustment_factor']:^12.4f} | {result['prediction']:^12.4f}")

print("\nMixing methods can significantly affect the adjustment factor!")

### 7.3 Impact of Governors

In [ ]:
# Compare governors for the same constraint and mixing
print("GOVERNOR COMPARISON\n")
print("Fixed: local_entropy + additive, n=12, b=3\n")
print(f"{'Governor':^30} | {'Adj. Factor':^12} | {'Predicted E':^12}")
print("="*60)

for governor in ["uniform_distribution", "entropy_maximization", "novelty_seeking"]:
    result = predict_exploration(
        n=12, b=3,
        constraint="local_entropy",
        mixing="additive",
        governor=governor
    )
    print(f"{governor:^30} | {result['adjustment_factor']:^12.4f} | {result['prediction']:^12.4f}")

print("\nGovernors also affect the effective constraint pressure!")

### 7.4 Full Architecture Comparison Matrix

In [ ]:
# Create heatmap of predictions across architectures
import matplotlib.pyplot as plt

constraints = ["sum_modulation", "local_entropy"]
mixings = ["additive", "multiplicative", "triple_sum"]
governors = ["uniform_distribution", "entropy_maximization", "novelty_seeking"]

# Create matrix for n=12
results_matrix = []
labels = []

for constraint in constraints:
    for mixing in mixings:
        row = []
        for governor in governors:
            result = predict_exploration(
                n=12, b=3,
                constraint=constraint,
                mixing=mixing,
                governor=governor
            )
            row.append(result['prediction'])
        results_matrix.append(row)
        labels.append(f"{constraint[:7]}...\n{mixing[:5]}...")

# Plot heatmap
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(results_matrix, cmap='YlOrRd', aspect='auto', vmin=0, vmax=1)

# Set ticks and labels
ax.set_xticks(range(len(governors)))
ax.set_yticks(range(len(labels)))
ax.set_xticklabels([g.replace('_', '\n') for g in governors], fontsize=9)
ax.set_yticklabels(labels, fontsize=8)

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Exploration Efficiency', fontsize=11)

# Add values to cells
for i in range(len(labels)):
    for j in range(len(governors)):
        text = ax.text(j, i, f'{results_matrix[i][j]:.3f}',
                      ha="center", va="center", color="black", fontsize=8)

ax.set_title('Architecture Comparison Matrix (n=12)', fontsize=13, fontweight='bold')
ax.set_xlabel('Governor', fontsize=11)
ax.set_ylabel('Constraint + Mixing', fontsize=11)

plt.tight_layout()
plt.savefig('architecture_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("Different architectures produce different exploration efficiencies!")
print(f"Range: {np.min(results_matrix):.4f} to {np.max(results_matrix):.4f}")

<a id='visualizations'></a>
## 8. Comprehensive Visualizations

This section generates publication-quality visualizations for the framework.

### 8.1 Model Performance Comparison

In [ ]:
# Performance metrics visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Class I metrics
metrics_class1 = ['R²', 'RMSE', 'MAE']
values_class1 = [0.95, 0.05, 0.04]
colors1 = ['#2E86AB', '#A23B72', '#F18F01']

bars1 = ax1.bar(metrics_class1, values_class1, color=colors1, alpha=0.7, edgecolor='black')
ax1.set_ylabel('Score', fontsize=12)
ax1.set_title('Class I Performance\n(CPR Sigmoid Model)', fontsize=13, fontweight='bold')
ax1.set_ylim(0, 1)
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, val in zip(bars1, values_class1):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{val:.3f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Class II metrics
metrics_class2 = ['R²', 'RMSE', 'MAE']
values_class2 = [0.99, 0.02, 0.015]
colors2 = ['#2E86AB', '#A23B72', '#F18F01']

bars2 = ax2.bar(metrics_class2, values_class2, color=colors2, alpha=0.7, edgecolor='black')
ax2.set_ylabel('Score', fontsize=12)
ax2.set_title('Class II Performance\n(Complexity Linear Model)', fontsize=13, fontweight='bold')
ax2.set_ylim(0, 1)
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, val in zip(bars2, values_class2):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{val:.3f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('model_performance.png', dpi=300, bbox_inches='tight')
plt.show()

print("Both models achieve excellent performance!")
print("Class II (complexity) model is slightly more accurate (R² = 0.99 vs 0.95)")

<a id='advanced'></a>
## 9. Advanced Topics

### 9.1 Sensitivity Analysis

In [ ]:
# Analyze sensitivity to system parameters
print("SENSITIVITY ANALYSIS: Effect of Base (b)\n")

n_fixed = 10
print(f"Fixed: n={n_fixed}, sum_modulation + additive + uniform_distribution\n")
print(f"{'Base (b)':>8} | {'CPR':>15} | {'Predicted E':>12} | {'% Change':>10}")
print("="*55)

prev_e = None
for b in [2, 3, 4, 5]:
    result = predict_exploration(
        n=n_fixed, b=b,
        constraint="sum_modulation",
        mixing="additive",
        governor="uniform_distribution"
    )
    
    e = result['prediction']
    cpr = result['adjusted_cpr']
    
    if prev_e is not None:
        pct_change = ((e - prev_e) / prev_e) * 100
        pct_str = f"{pct_change:+.2f}%"
    else:
        pct_str = "--"
    
    print(f"{b:8d} | {cpr:15.2e} | {e:12.4f} | {pct_str:>10}")
    prev_e = e

print("\nHigher base → Lower CPR → Higher exploration")

### 9.2 Critical Point Analysis

In [ ]:
# Find the critical point (steepest slope)
n_range = np.linspace(6, 16, 100)
e_values = []

for n in n_range:
    result = predict_exploration(
        n=int(np.round(n)), b=3,
        constraint="sum_modulation",
        mixing="additive",
        governor="uniform_distribution"
    )
    e_values.append(result['prediction'])

# Calculate derivatives
de_dn = np.gradient(e_values, n_range)
d2e_dn2 = np.gradient(de_dn, n_range)

# Find inflection point (where second derivative is zero)
critical_idx = np.argmax(de_dn)
critical_n = n_range[critical_idx]
critical_e = e_values[critical_idx]

# Plot
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 4))

# Original function
ax1.plot(n_range, e_values, linewidth=2, color='#2E86AB')
ax1.plot(critical_n, critical_e, 'r*', markersize=15, label=f'Critical point')
ax1.set_xlabel('System Size (n)', fontsize=11)
ax1.set_ylabel('Exploration Efficiency', fontsize=11)
ax1.set_title('E(n)', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()

# First derivative
ax2.plot(n_range, de_dn, linewidth=2, color='#F18F01')
ax2.plot(critical_n, de_dn[critical_idx], 'r*', markersize=15, label='Maximum')
ax2.set_xlabel('System Size (n)', fontsize=11)
ax2.set_ylabel('dE/dn', fontsize=11)
ax2.set_title('First Derivative', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.legend()

# Second derivative
ax3.plot(n_range, d2e_dn2, linewidth=2, color='#A23B72')
ax3.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax3.set_xlabel('System Size (n)', fontsize=11)
ax3.set_ylabel('d²E/dn²', fontsize=11)
ax3.set_title('Second Derivative (Curvature)', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('critical_point_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Critical Point: n ≈ {critical_n:.2f}")
print(f"Maximum growth rate: dE/dn ≈ {de_dn[critical_idx]:.4f}")

<a id='applications'></a>
## 10. Real-World Applications

### 10.1 System Design: Targeting Specific Exploration Levels

In [ ]:
# Design a system to achieve target exploration
def design_system_for_target_e(target_e: float, b: int = 3, 
                               constraint: str = "sum_modulation",
                               mixing: str = "additive",
                               governor: str = "uniform_distribution",
                               tolerance: float = 0.01) -> int:
    """
    Find system size n that achieves target exploration efficiency.
    """
    # Binary search
    n_min, n_max = 4, 20
    
    while n_max - n_min > 1:
        n_mid = (n_min + n_max) // 2
        result = predict_exploration(
            n=n_mid, b=b,
            constraint=constraint,
            mixing=mixing,
            governor=governor
        )
        e_mid = result['prediction']
        
        if abs(e_mid - target_e) < tolerance:
            return n_mid
        elif e_mid < target_e:
            n_max = n_mid
        else:
            n_min = n_mid
    
    return n_mid

# Design examples
print("SYSTEM DESIGN FOR TARGET EXPLORATION\n")
print(f"{'Target E':>10} | {'Designed n':>12} | {'Actual E':>10} | {'Error':>10}")
print("="*50)

for target in [0.1, 0.3, 0.5, 0.7, 0.85]:
    n_designed = design_system_for_target_e(target)
    result = predict_exploration(
        n=n_designed, b=3,
        constraint="sum_modulation",
        mixing="additive",
        governor="uniform_distribution"
    )
    actual_e = result['prediction']
    error = abs(actual_e - target)
    
    print(f"{target:10.2f} | {n_designed:12d} | {actual_e:10.4f} | {error:10.4f}")

print("\nUse Case: Design systems with specific exploration requirements!")

### 10.2 Comparative Analysis: Choosing the Best Architecture

In [ ]:
# Compare all architectures for a given system size
def compare_all_architectures(n: int = 12, b: int = 3, top_k: int = 5):
    """
    Compare all available architectures and rank by exploration.
    """
    constraints = ["sum_modulation", "local_entropy"]
    mixings = ["additive", "multiplicative", "triple_sum"]
    governors = ["uniform_distribution", "entropy_maximization", "novelty_seeking"]
    
    results = []
    
    for constraint in constraints:
        for mixing in mixings:
            for governor in governors:
                result = predict_exploration(
                    n=n, b=b,
                    constraint=constraint,
                    mixing=mixing,
                    governor=governor
                )
                
                results.append({
                    'constraint': constraint,
                    'mixing': mixing,
                    'governor': governor,
                    'exploration': result['prediction'],
                    'adjustment': result['adjustment_factor']
                })
    
    # Sort by exploration
    results.sort(key=lambda x: x['exploration'], reverse=True)
    
    return results

# Run comparison
print("ARCHITECTURE RANKING (n=12, b=3)\n")
print("Top 5 Highest Exploration:")
print(f"{'Rank':>4} | {'E':>8} | {'Architecture':^60}")
print("="*80)

all_results = compare_all_architectures()

for i, res in enumerate(all_results[:5], 1):
    arch_str = f"{res['constraint']} + {res['mixing']} + {res['governor']}"
    print(f"{i:4d} | {res['exploration']:8.4f} | {arch_str}")

print("\nBottom 5 Lowest Exploration:")
print(f"{'Rank':>4} | {'E':>8} | {'Architecture':^60}")
print("="*80)

for i, res in enumerate(all_results[-5:], len(all_results)-4):
    arch_str = f"{res['constraint']} + {res['mixing']} + {res['governor']}"
    print(f"{i:4d} | {res['exploration']:8.4f} | {arch_str}")

print("\nUse Case: Select optimal architecture for your exploration goals!")

### 10.3 Prediction Before Expensive Simulation

In [ ]:
# Use case: Rapid prototyping without running full simulations
print("USE CASE: Rapid Prototyping\n")
print("Scenario: You want to explore different system configurations")
print("without running expensive simulations.\n")
print("="*70)

candidate_configs = [
    {"n": 10, "b": 3, "constraint": "sum_modulation", 
     "mixing": "additive", "governor": "uniform_distribution",
     "description": "Baseline configuration"},
    {"n": 12, "b": 3, "constraint": "sum_modulation", 
     "mixing": "multiplicative", "governor": "entropy_maximization",
     "description": "Larger system with entropy maximization"},
    {"n": 14, "b": 3, "constraint": "local_entropy", 
     "mixing": "triple_sum", "governor": "novelty_seeking",
     "description": "Maximum exploration setup"},
    {"n": 8, "b": 4, "constraint": "sum_modulation", 
     "mixing": "additive", "governor": "uniform_distribution",
     "description": "Higher base, smaller system"},
]

print("Evaluating {} candidate configurations...\n".format(len(candidate_configs)))

for i, config in enumerate(candidate_configs, 1):
    print(f"Config {i}: {config['description']}")
    
    result = predict_exploration(
        n=config['n'],
        b=config['b'],
        constraint=config['constraint'],
        mixing=config['mixing'],
        governor=config['governor']
    )
    
    print(f"  Parameters: n={config['n']}, b={config['b']}")
    print(f"  Architecture: {config['constraint']} + {config['mixing']} + {config['governor']}")
    print(f"  → Predicted Exploration: {result['prediction']:.4f}")
    print(f"  → CPR: {result['adjusted_cpr']:.2e}")
    print()

print("\nBenefit: Instantly evaluate configurations without simulation!")
print("Simulation time saved: Hours → Seconds")

## Summary and Conclusions

### Key Takeaways

1. **Two Universality Classes**: Constraints exhibit two fundamentally different behaviors
   - Class I (density-based): Sigmoid phase transition
   - Class II (structure-based): Linear complexity scaling

2. **High Accuracy**: Both models achieve excellent performance
   - Class I: R² > 0.95
   - Class II: R² > 0.99

3. **Complete Coverage**: 100% of 27 architectures successfully predicted

4. **Three Regimes**: Systems transition through constrained, critical, and emergent regimes

5. **Practical Applications**:
   - Design systems for target exploration levels
   - Compare architectures without simulation
   - Understand constraint impacts
   - Optimize system configurations

### Next Steps

- Explore the interactive demo: `python demo.py`
- Read the technical report: `TECHNICAL_REPORT_SOLUTION.md`
- Review validation results: `SCIENTIFIC_VALIDATION_REPORT.md`
- Check implementation: `implementation_complete.py`

### Framework Citation

If you use this framework in your research, please cite:

```
CPR Framework: A Unified Theory of Constraint Universality Classes
Discovery of two fundamental universality classes in constrained dynamical systems
```